In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import xlwings as xl
import regex as re
import os
from pathlib import Path


edge_options = webdriver.EdgeOptions()
prefs = {'download.default_directory' : r'D:\Masscom\Data\CareerLink'}
edge_options.add_experimental_option('prefs', prefs)


In [2]:
ten_dang_nhap = ''
mat_khau = ""

#------------------------------------------
driver = webdriver.Edge(options = edge_options)
driver.get(r"https://www.careerlink.vn/nha-tuyen-dung/login")
wait = WebDriverWait(driver, 6)
time.sleep(2)
driver.find_element_by_id(r'_username').send_keys(ten_dang_nhap)
driver.find_element_by_id(r'_password').send_keys(mat_khau)


#sign in
wait.until(EC.element_to_be_clickable((By.NAME,'btnCompanyLogin'))).click()

#thư xin việc đã nhận
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#applications-link>a'))).click()

#job dropdown
wait.until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[4]/div[1]/div/button'))).click()

#job element list
jobs1 = driver.find_elements_by_class_name("dropdown-item")
jobs = jobs1[6:len(jobs1)-1]
real_jobs = [i for i in jobs if i.get_attribute("textContent")[-3:] != "(0)"]


#job_links
job_links = []
for i in real_jobs:
    job_links.append(i.get_attribute('href'))

#open job link in new tab, click download button then close the new tab
for i in range(len(job_links)):
    driver.execute_script("window.open()")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(job_links[i])
    WebDriverWait(driver, 6).until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[4]/div[2]/div/div[2]/div/a'))).click()
    print(i+1)
    time.sleep(10)
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(5)



C:\Users\Admin\AppData\Local\Temp/ipykernel_13844/4225382121.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id(r'_username').send_keys(ten_dang_nhap)
C:\Users\Admin\AppData\Local\Temp/ipykernel_13844/4225382121.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id(r'_password').send_keys(mat_khau)
C:\Users\Admin\AppData\Local\Temp/ipykernel_13844/4225382121.py:23: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  jobs1 = driver.find_elements_by_class_name("dropdown-item")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


In [2]:
path = os.getcwd()
files = sorted(Path(r'D:\Masscom\Data\CareerLink').iterdir(), key=os.path.getmtime)

print(files)


header = 'Stt	Ngày nhận hồ sơ	Ngày nhận HS 	Tháng nhận HS	Năm nhận HS	Công ty	Đơn vị / Chi nhánh	Khối/TT	Phòng ban/Bộ phận	Vị trí	Họ và tên	Ngày tháng năm sinh	Số điện thoại	Email	Tuổi	Giới tính	Tốt nghiệp trường	Trình độ cao nhất	Địa chỉ	Nguồn	Hình thức	Loại kênh	Kênh TD	KQ hồ sơ'
header = header.split('\t')
final_df = pd.DataFrame(columns = header)


for i in range(len(files)):
    df = pd.read_excel("%s"%(files[i]))
    df1 = pd.DataFrame(columns = header)

    df1["Họ và tên"] = df.iloc[5:,1]
    df1["Ngày nhận hồ sơ"] = pd.to_datetime(df.iloc[5:,2], format = r"%d/%m/%Y %H:%M:%S")
    df1['Ngày nhận hồ sơ'] = df1['Ngày nhận hồ sơ'].dt.date
    df1["Số điện thoại"] = df.iloc[5:,4]
    df1["Email"] = df.iloc[5:,3]
    df1["Ngày tháng năm sinh"] = pd.to_datetime(df.iloc[5:,6]).dt.year
    df1['Ngày nhận hồ sơ'] = pd.to_datetime(df1['Ngày nhận hồ sơ'])
    df1['Ngày nhận HS '] = df1['Ngày nhận hồ sơ'].dt.day
    df1['Tháng nhận HS'] = df1['Ngày nhận hồ sơ'].dt.month
    df1['Năm nhận HS'] = df1['Ngày nhận hồ sơ'].dt.year
    df1['Hình thức'] = 'Online'
    df1['Nguồn'] = 'Bên ngoài'
    df1['Loại kênh'] = 'Website'
    df1['Kênh TD'] = 'CareerLink'
    try:
        df1 = df1.astype({"Ngày tháng năm sinh": int})
        df1['Tuổi'] = df1['Năm nhận HS'] - df1['Ngày tháng năm sinh']
    except: 
        df1['Tuổi'] = 'zzzzzzz'


    df1['Ngày tháng năm sinh'] = df1['Ngày tháng năm sinh'].fillna('zzzzzzzzzzz')

    df1["Vị trí"] = df.iloc[[0],[1]].values[0][0]

    df1['Giới tính'] = ''
    df1.loc[df['Unnamed: 5'] == 'Male', 'Giới tính'] = 'Nam'
    df1.loc[df['Unnamed: 5'] == 'Female', 'Giới tính'] = 'Nữ'

    final_df = final_df.append(df1)

#send data to excel
book = xl.books.open(r"D:\Masscom\Data\2022_Data Tuyen dung.xlsx")
sheet = book.sheets("CareerLink")
sheet["A2"].options(pd.DataFrame,index = False, header = False).value = final_df

[WindowsPath('D:/Masscom/Data/CareerLink/applications_for_job_2249692_05_05_2022.xlsx'), WindowsPath('D:/Masscom/Data/CareerLink/applications_for_job_2249686_05_05_2022.xlsx'), WindowsPath('D:/Masscom/Data/CareerLink/applications_for_job_2249866_05_05_2022.xlsx'), WindowsPath('D:/Masscom/Data/CareerLink/applications_for_job_2255401_05_05_2022.xlsx'), WindowsPath('D:/Masscom/Data/CareerLink/applications_for_job_2255457_05_05_2022.xlsx'), WindowsPath('D:/Masscom/Data/CareerLink/applications_for_job_2263046_05_05_2022.xlsx'), WindowsPath('D:/Masscom/Data/CareerLink/applications_for_job_2243740_05_05_2022.xlsx'), WindowsPath('D:/Masscom/Data/CareerLink/applications_for_job_2243350_05_05_2022.xlsx'), WindowsPath('D:/Masscom/Data/CareerLink/applications_for_job_2262500_05_05_2022.xlsx'), WindowsPath('D:/Masscom/Data/CareerLink/applications_for_job_2262589_05_05_2022.xlsx'), WindowsPath('D:/Masscom/Data/CareerLink/applications_for_job_2238620_05_05_2022.xlsx'), WindowsPath('D:/Masscom/Data/Ca